In [ ]:
from fuzzywuzzy import process, fuzz
from rapidfuzz import process, fuzz
from tqdm import tqdm

import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# importing the datasets
saudi_data = pd.read_excel("Data/SFDAData.xlsx")
usfda_data = pd.read_excel("Data/USData/USPoliData.xlsx")
ema_data = pd.read_excel("Data/EMAData/EMA_cleaned.xlsx")
UK_data = pd.read_excel("Data/UKData/Cleaned_UK.xlsx")


In [ ]:
# loop through the saudi and UK data to put _ in the column names where there is a space
for col in saudi_data.columns:
    if " " in col:
        saudi_data.rename(columns={col: col.replace(" ", "_")}, inplace=True)

for col in UK_data.columns:
    if " " in col:
        UK_data.rename(columns={col: col.replace(" ", "_")}, inplace=True)

# rename Approval_Year to be US_Approval_Year
usfda_data.rename(columns={"Approval_Year": "US_Approval_Year"}, inplace=True)

# First Approval Date in the UK to be UK_Approval_Year
UK_data.rename(columns={"First_Approval_Date": "UK_Approval_Year"}, inplace=True)


In [ ]:
# show column names
print(saudi_data.columns)
print(usfda_data.columns)
print(ema_data.columns)
print(UK_data.columns)

Index(['National_Drug_ID', 'ReferenceNumber', 'Old_register_Number',
       'Registration_Year', 'DrugType', 'Sub-Type', 'Scientific_Name',
       'Trade_Name', 'Strength', 'StrengthUnit', 'PharmaceuticalForm',
       'AdministrationRoute', 'AtcCode1', 'AtcCode2', 'Size', 'SizeUnit',
       'PackageTypes', 'PackageSize', 'Legal_Status', 'Product_Control',
       'Distribute_area', 'Public_price', 'Marketing_Company',
       'Marketing_Country', 'Manufacture_Name', 'Manufacture_Country',
       'Secondry_package__manufacture', 'Main_Agent', 'Secosnd_Agent',
       'Third_agent', 'Description_Code', 'Authorization_Status',
       'International_Drug_ID', 'POL_Brand_Name', 'POL_Company_(Parent)',
       'POL_Dosage_Form', 'POL_Strength_Comment', 'POL_Company_(Subsidiary)',
       'POL_Drug_Type', 'POL_Hospital_Status', 'POL_Parallel_Import',
       'EMA_Brand_ID', 'POL_Therapy_Area_/_Indication', 'POL_ATC_Code',
       'POL_First_Price_Date', 'POL_First_Price_(LCU)',
       'POL_First_Pri

In [ ]:
# Convert US_Approval_Year to datetime and extract the year
usfda_data["US_Approval_Year"] = pd.to_datetime(
    usfda_data["US_Approval_Year"], format="%Y", errors="coerce"
).dt.year

# Convert UK_Approval_Year to datetime and extract the year
UK_data["UK_Approval_Year"] = pd.to_datetime(
    UK_data["UK_Approval_Year"], errors="coerce"
).dt.year

In [ ]:
# drop any rows in the usfdadata where the US_Approval_Year is 1900
usfda_data = usfda_data[usfda_data.US_Approval_Year != 1900]

In [7]:
# total of rows in each dataset
print("Total rows in Saudi Arabia dataset: ", len(saudi_data))
print("Total rows in US FDA dataset: ", len(usfda_data))
print("Total rows in EMA dataset: ", len(ema_data))
print("Total rows in UK dataset: ", len(UK_data))

Total rows in Saudi Arabia dataset:  1108
Total rows in US FDA dataset:  27914
Total rows in EMA dataset:  2105
Total rows in UK dataset:  4784


In [ ]:
# how many had EMA_Bran_ID in the saudi data?
print(
    "Total rows in Saudi Arabia dataset with EMA_Brand_ID: ",
    len(saudi_data[saudi_data["EMA_Brand_ID"].notnull()]),
)

Total rows in Saudi Arabia dataset with EMA_Brand_ID:  647


# 1- Match with EMA 

In [9]:
# Perform a left merge to add EMA_Approval_Year to saudi_data based on EMA_Brand_ID
saudi_data = pd.merge(
    saudi_data,
    ema_data[["EMA_Brand_ID", "EMA_Approval_Year"]],
    on="EMA_Brand_ID",
    how="left",
)

In [10]:
# how many rows have been merged between saudi and EMA data (how many EMA years added)
print(saudi_data["EMA_Approval_Year"].count())

645


---

# 2- Match with US

2 uniqe identifiers exist but they need cleaning before merging with the Saudi Data. Given the volume of th data, we'll only work with those who have non missing for either of these identifiers. 

Upon inspection, looks like we have many duplicates by International_Drug_ID and EMA_Brand_ID in the data. We'll drop any duplicates by International_Drug_ID or EMA_Brand_ID that have the same US_Approval_Year


In [ ]:
# how many in the usfda_data have International_Drug_ID and how many have EMA_Brand_ID?
print(
    "Total rows in US FDA dataset with International_Drug_ID: ",
    len(usfda_data[usfda_data["International_Drug_ID"].notnull()]),
)
print(
    "Total rows in US FDA dataset with EMA_Brand_ID: ",
    len(usfda_data[usfda_data["EMA_Brand_ID"].notnull()]),
)

Total rows in US FDA dataset with International_Drug_ID:  2738
Total rows in US FDA dataset with EMA_Brand_ID:  2605


> ## First inspect data and duplicates before merging

## Duplicates by the international drug ID in the saudi data

In [ ]:
# only keep rows that have either International_Drug_ID or EMA_Brand_ID in the usfda_data
usfda_data = usfda_data[
    usfda_data["International_Drug_ID"].notnull() | usfda_data["EMA_Brand_ID"].notnull()
]

In [ ]:
# How many duplicates by the International_Drug_ID in the saudi data
print(
    "Total duplicates in Saudi Arabia dataset by International Drug ID: ",
    saudi_data.duplicated(subset="International_Drug_ID").sum(),
)

# how many of the duplicates also have the same Registration_Year
print(
    "Total duplicates in Saudi Arabia dataset by International Drug ID and Registration Year: ",
    saudi_data.duplicated(subset=["International_Drug_ID", "Registration_Year"]).sum(),
)
print(saudi_data["International_Drug_ID"].nunique())


# convert the International_Drug_ID to string in both saudi and usfda data
saudi_data["International_Drug_ID"] = saudi_data["International_Drug_ID"].astype(str)
usfda_data["International_Drug_ID"] = usfda_data["International_Drug_ID"].astype(str)

Total duplicates in Saudi Arabia dataset by International Drug ID:  346
Total duplicates in Saudi Arabia dataset by International Drug ID and Registration Year:  336
761


In [ ]:
# How many have ema_brand_id in the usfda_data?
print(
    "Total rows in US FDA dataset with EMA_Brand_ID: ",
    len(usfda_data[usfda_data["EMA_Brand_ID"].notnull()]),
)

# How many have International_Drug_ID in the usfda_data?
print(
    "Total rows in US FDA dataset with International_Drug_ID: ",
    len(usfda_data[usfda_data["International_Drug_ID"].notnull()]),
)


# how many have ema_brand_id but not International_Drug_ID in the usfda_data?
print(
    "Total rows in US FDA dataset with EMA_Brand_ID but not International_Drug_ID: ",
    len(
        usfda_data[
            (usfda_data["EMA_Brand_ID"].notnull())
            & (usfda_data["International_Drug_ID"].isnull())
        ]
    ),
)

# how many have International_Drug_ID but not ema_brand_id in the usfda_data?
print(
    "Total rows in US FDA dataset with International_Drug_ID but not EMA_Brand_ID: ",
    len(
        usfda_data[
            (usfda_data["International_Drug_ID"].notnull())
            & (usfda_data["EMA_Brand_ID"].isnull())
        ]
    ),
)

Total rows in US FDA dataset with EMA_Brand_ID:  2605
Total rows in US FDA dataset with International_Drug_ID:  3672
Total rows in US FDA dataset with EMA_Brand_ID but not International_Drug_ID:  0
Total rows in US FDA dataset with International_Drug_ID but not EMA_Brand_ID:  1067


In [ ]:
# how many uniqe International_Drug_ID in the usfda_data
print(
    "Total unique International_Drug_ID in US FDA dataset: ",
    len(usfda_data["International_Drug_ID"].unique()),
)

# how many uniqe EMA_Brand_ID in the usfda_data
print(
    "Total unique EMA_Brand_ID in US FDA dataset: ",
    len(usfda_data["EMA_Brand_ID"].unique()),
)


Total unique International_Drug_ID in US FDA dataset:  2170
Total unique EMA_Brand_ID in US FDA dataset:  687


In [ ]:
# How many in the usfda_data are duplicates by International_Drug_ID and EMA_Brand_ID
# and also happen to have the same US_Approval_Year?

usfda_data["International_Drug_ID"] = usfda_data["International_Drug_ID"].str.strip()
usfda_data["EMA_Brand_ID"] = usfda_data["EMA_Brand_ID"].str.strip()


print(
    "Total rows in US FDA dataset that are duplicates by International_Drug_ID that have the same US_Approval_Year: ",
    len(
        usfda_data[
            usfda_data.duplicated(
                subset=["International_Drug_ID", "US_Approval_Year"], keep=False
            )
        ]
    ),
)

print(
    "Total rows in US FDA dataset that are duplicates by EMA_Brand_ID that have the same US_Approval_Year: ",
    len(
        usfda_data[
            usfda_data.duplicated(
                subset=["EMA_Brand_ID", "US_Approval_Year"], keep=False
            )
        ]
    ),
)

Total rows in US FDA dataset that are duplicates by International_Drug_ID that have the same US_Approval_Year:  1682
Total rows in US FDA dataset that are duplicates by EMA_Brand_ID that have the same US_Approval_Year:  3429


## Perform the merge after dropping the duplicates

### Column 1: Based on `International_Drug_ID`

The below method keeps only one version for those with the same International ID and Year. as for those with International ID dueps but diffrent approval years, they will be concatenated in one cell. this is ideal for when we don't won't to drop any valueable info during automation untill we douple check later. we have 212 of those dupes with diffrent years that remain undropped.

In [ ]:
# Drop exact duplicates based on International_Drug_ID and US_Approval_Year
uniqe_international_usfda_data = usfda_data.drop_duplicates(
    subset=["International_Drug_ID", "US_Approval_Year"], keep="first"
)

# Group by International_Drug_ID and concatenate approval years into a single entry
uniqe_international_usfda_data = uniqe_international_usfda_data.groupby(
    "International_Drug_ID", as_index=False
).agg(
    {
        "US_Approval_Year": lambda x: ", ".join(
            map(str, sorted(set(x)))
        )  # Joins years into one entry
    }
)

# Ensure US_Approval_Year is a clean string (removes unnecessary spaces)
uniqe_international_usfda_data["US_Approval_Year"] = (
    uniqe_international_usfda_data["US_Approval_Year"].astype(str).str.strip()
)

# Check how many unique drug IDs remain
print(
    "Total unique drugs after grouping by International_Drug_ID:",
    uniqe_international_usfda_data.shape[0],
)

Total unique drugs after grouping by International_Drug_ID: 2170


In [ ]:
# Strip spaces from 'International_Drug_ID' in both datasets
uniqe_international_usfda_data["International_Drug_ID"] = (
    uniqe_international_usfda_data["International_Drug_ID"].str.strip()
)
saudi_data["International_Drug_ID"] = saudi_data["International_Drug_ID"].str.strip()

### Final check before performing the merge!!

In [ ]:
duplicate_check = uniqe_international_usfda_data.duplicated(
    subset=["International_Drug_ID"], keep=False
).sum()
print(f"Duplicates in uniqe_international_usfda_data after cleaning: {duplicate_check}")

### PERFECT!

Duplicates in uniqe_international_usfda_data after cleaning: 0


In [ ]:
# Perform a left merge to add US Approval Year to the Saudi Data based on International_Drug_ID
saudi_data = pd.merge(
    saudi_data,
    uniqe_international_usfda_data[["International_Drug_ID", "US_Approval_Year"]],
    on="International_Drug_ID",
    how="left",
)

saudi_data = saudi_data.rename(
    columns={"US_Approval_Year": "US_Approval_Year_International"}
)


### Column 2: Based on `EMA_Brand_ID`

In [ ]:
# Drop exact duplicates based on EMA_ID and US_Approval_Year
uniqe_EMAID_usfda_data = usfda_data.drop_duplicates(
    subset=["EMA_Brand_ID", "US_Approval_Year"], keep="first"
)

# Group by EMA ID and concatenate approval years into a single entry
uniqe_EMAID_usfda_data = uniqe_EMAID_usfda_data.groupby(
    "EMA_Brand_ID", as_index=False
).agg(
    {
        "US_Approval_Year": lambda x: ", ".join(
            map(str, sorted(set(x)))
        )  # Joins years into one entry
    }
)

# Ensure US_Approval_Year is a clean string (removes unnecessary spaces)
uniqe_EMAID_usfda_data["US_Approval_Year"] = (
    uniqe_EMAID_usfda_data["US_Approval_Year"].astype(str).str.strip()
)

# Check how many unique drug IDs remain
print(
    "Total unique drugs after grouping by EMA_Brand_ID:",
    uniqe_EMAID_usfda_data.shape[0],
)

Total unique drugs after grouping by EMA_Brand_ID: 686


In [ ]:
# Perform a left merge to add US Approval Year to saudi_data based on EMA_Brand_ID
saudi_data = pd.merge(
    saudi_data,
    uniqe_EMAID_usfda_data[["EMA_Brand_ID", "US_Approval_Year"]],
    on="EMA_Brand_ID",
    how="left",
)

saudi_data = saudi_data.rename(columns={"US_Approval_Year": "US_Approval_Year_EMA"})

---

# 3- Match with UK

Here we'll repeat the same methods we did for the US data

In [ ]:
# only keep rows that have either International_Drug_ID or EMA_Brand_ID in the usfda_data
UK_data = UK_data[
    UK_data["International_Drug_ID"].notnull() | UK_data["EMA_Brand_ID"].notnull()
]


In [ ]:
# how many in the UK data have International_Drug_ID and how many have EMA_Brand_ID?
print(
    "Total rows in US FDA dataset with International_Drug_ID: ",
    len(UK_data[UK_data["International_Drug_ID"].notnull()]),
)
print(
    "Total rows in US FDA dataset with EMA_Brand_ID: ",
    len(UK_data[UK_data["EMA_Brand_ID"].notnull()]),
)

Total rows in US FDA dataset with International_Drug_ID:  2714
Total rows in US FDA dataset with EMA_Brand_ID:  2311


In [ ]:
# convert to string and remove and leading or trailing spaces
UK_data["International_Drug_ID"] = UK_data["International_Drug_ID"].astype(str)
UK_data["EMA_Brand_ID"] = UK_data["EMA_Brand_ID"].astype(str)


UK_data["International_Drug_ID"] = UK_data["International_Drug_ID"].str.strip()
UK_data["EMA_Brand_ID"] = UK_data["EMA_Brand_ID"].str.strip()

In [ ]:
# How many in the UK data are duplicates by International_Drug_ID and EMA_Brand_ID
# and also happen to have the same UK_Approval_Year?

print(
    "Total rows in UK dataset that are duplicates by International_Drug_ID that have the same UK_Approval_Year: ",
    len(
        UK_data[
            UK_data.duplicated(
                subset=["International_Drug_ID", "UK_Approval_Year"], keep=False
            )
        ]
    ),
)

print(
    "Total rows in UK dataset that are duplicates by EMA_Brand_ID that have the same UK_Approval_Year: ",
    len(
        UK_data[
            UK_data.duplicated(subset=["EMA_Brand_ID", "UK_Approval_Year"], keep=False)
        ]
    ),
)

Total rows in UK dataset that are duplicates by International_Drug_ID that have the same UK_Approval_Year:  0
Total rows in UK dataset that are duplicates by EMA_Brand_ID that have the same UK_Approval_Year:  2310


## Perform the merge after dropping the duplicates

### Column 1: Based on `International_Drug_ID`

In [ ]:
# Drop exact duplicates based on International_Drug_ID and UK_Approval_Year
uniqe_international_UK_data = UK_data.drop_duplicates(
    subset=["International_Drug_ID", "UK_Approval_Year"], keep="first"
)

# Group by International_Drug_ID and concatenate approval years into a single entry
uniqe_international_UK_data = uniqe_international_UK_data.groupby(
    "International_Drug_ID", as_index=False
).agg(
    {
        "UK_Approval_Year": lambda x: ", ".join(
            map(str, sorted(set(x)))
        )  # Joins years into one entry
    }
)

# Ensure UK_Approval_Year is a clean string (removes unnecessary spaces)
uniqe_international_UK_data["UK_Approval_Year"] = (
    uniqe_international_UK_data["UK_Approval_Year"].astype(str).str.strip()
)

# Check how many unique drug IDs remain
print(
    "Total unique drugs after grouping by International_Drug_ID:",
    uniqe_international_UK_data.shape[0],
)

Total unique drugs after grouping by International_Drug_ID: 2709


In [ ]:
# Strip spaces from 'International_Drug_ID' in both datasets
uniqe_international_UK_data["International_Drug_ID"] = uniqe_international_UK_data[
    "International_Drug_ID"
].str.strip()
saudi_data["International_Drug_ID"] = saudi_data["International_Drug_ID"].str.strip()

In [ ]:
duplicate_check2 = uniqe_international_UK_data.duplicated(
    subset=["International_Drug_ID"], keep=False
).sum()
print(f"Duplicates in uniqe_international_usfda_data after cleaning: {duplicate_check}")

### PERFECT!

Duplicates in uniqe_international_usfda_data after cleaning: 0


In [ ]:
# Perform a left merge to add International_Drug_ID to saudi_data based on International_Drug_ID
saudi_data = pd.merge(
    saudi_data,
    uniqe_international_UK_data[["International_Drug_ID", "UK_Approval_Year"]],
    on="International_Drug_ID",
    how="left",
)

saudi_data = saudi_data.rename(
    columns={"UK_Approval_Year": "UK_Approval_Year_International"}
)

### Column 2: Based on `EMA_Brand_ID`

In [ ]:
# Drop exact duplicates based on EMA_ID and UK_Approval_Year
uniqe_EMAID_UK_data = UK_data.drop_duplicates(
    subset=["EMA_Brand_ID", "UK_Approval_Year"], keep="first"
)

# Group by EMA ID and concatenate approval years into a single entry
uniqe_EMAID_UK_data = uniqe_EMAID_UK_data.groupby("EMA_Brand_ID", as_index=False).agg(
    {
        "UK_Approval_Year": lambda x: ", ".join(
            map(str, sorted(set(x)))
        )  # Joins years into one entry
    }
)

# Ensure UK_Approval_Year is a clean string (removes unnecessary spaces)
uniqe_EMAID_UK_data["UK_Approval_Year"] = (
    uniqe_EMAID_UK_data["UK_Approval_Year"].astype(str).str.strip()
)

# Check how many unique drug IDs remain
print(
    "Total unique drugs after grouping by EMA_Brand_ID:", uniqe_EMAID_UK_data.shape[0]
)

Total unique drugs after grouping by EMA_Brand_ID: 947


In [ ]:
# Perform a left merge to add EMA UK year to saudi_data based on EMA_Brand_ID
saudi_data = pd.merge(
    saudi_data,
    uniqe_EMAID_UK_data[["EMA_Brand_ID", "UK_Approval_Year"]],
    on="EMA_Brand_ID",
    how="left",
)

saudi_data = saudi_data.rename(columns={"UK_Approval_Year": "UK_Approval_Year_EMA"})

### Find out how may we matched over all

In [ ]:
# Count the number of rows that don't have any of the specified approval years
missing_approval_years = saudi_data[
    saudi_data["EMA_Approval_Year"].isnull()
    & saudi_data["US_Approval_Year_International"].isnull()
    & saudi_data["US_Approval_Year_EMA"].isnull()
    & saudi_data["UK_Approval_Year_International"].isnull()
    & saudi_data["UK_Approval_Year_EMA"].isnull()
]

# Print the number of such rows
print("Number of rows without any approval years:", len(missing_approval_years))

Number of rows without any approval years: 134


In [ ]:
# Create a new column to store the fuzzy matching results
saudi_data["fuzzy_matched"] = None

# Now iterate through the Saudi data and perform fuzzy matching with US data
for index, saudi_row in saudi_data.iterrows():
    best_match = None
    highest_score = 0

    for _, us_row in usfda_data.iterrows():
        # Combine the fields for comparison
        saudi_combined = f"{saudi_row['Brand_Name']} {saudi_row['Strength']} {saudi_row['Applicant']}"
        us_combined = (
            f"{us_row['Brand_Name']} {us_row['Strength']} {us_row['Applicant']}"
        )

        # Calculate the fuzzy match score
        score = fuzz.token_sort_ratio(saudi_combined, us_combined)

        # Update the best match if the score is higher
        if score > highest_score:
            highest_score = score
            best_match = us_row["US_Approval_Year"]

    # Add the best match to the new column if the score is above a threshold
    saudi_data.at[index, "fuzzy_matched"] = best_match if highest_score > 80 else None

In [ ]:
# Count the number of rows where fuzzy_matched is not null
fuzzy_matched_count = saudi_data["fuzzy_matched"].notnull().sum()

# Print the count
print("Number of rows fuzzy matched:", fuzzy_matched_count)


only 134 out of 1108 have no match!

In [ ]:
# save the saudi data to a new excel file
saudi_data.to_excel("Data/Merged_Data.xlsx", index=False)